In [1]:
using Revise
import SSD

Precompiling SSD
  ✓ SSD
  1 dependency successfully precompiled in 9 seconds. 66 already precompiled.


In [2]:
using CPUTime
using CSV
using DataFrames
using Dates
using DelimitedFiles
using Distributions
using Distances
using Gurobi
using LinearAlgebra
using Plots
using Random

In [ ]:
using TimerOutputs

# Create a TimerOutput, this is the main type that keeps track of everything.
const to = TimerOutput()

In [ ]:
function get_res(file)
    extract = false
    instance = ""
    method = ""
    lb = -99
    ub = -99
    cpu_t = -99
    status = ""
    iter = -99
    mess = ""
    backup = ""
    i=1
    open(file,"r") do f  
        while !eof(f)
            line = readline(f)
            sline = split(line)
            if i == 7
                backup = line
            end
            if size(sline)[1] > 0
                if extract==true
                    if sline[1] == "Instance:"
                        instance = split(split(sline[2], "/")[end],".")[1]
                    elseif sline[1] == "Method:"
                        method = sline[2]
                    elseif sline[1] == "LB:"
                        lb = parse(Float64,sline[2])
                    elseif sline[1] == "UB:"
                        ub = parse(Float64,sline[2])
                    elseif sline[1] == "CPU_t:"
                        cpu_t = parse(Float64,sline[2])
                    elseif sline[1] == "Status:"
                        status = sline[2]
                    elseif sline[1] == "Iter:"
                        iter = parse(Float64,sline[2])
                    end
                end
                if sline[1] == "RESULTS"                
                    extract = true
                end
                if sline[1] == "slurmstepd:"
                    println(file)
                    mess = join(sline[end-2:end-1]," ")
                    if backup != ""
                        bk = split(backup,":")
                        instance = split(bk[2],",")[1]
                        method=bk[end]
                        println(bk)
                        println(instance)
                        println(method)
                    end
                    return instance, method, lb, ub, cpu_t, status, iter, mess
                end
            end
            i+=1
        end
    end
    return instance, method, lb, ub, cpu_t, status, iter, mess
end

In [ ]:
df_res1 = DataFrame(inst=String[], method=String[], lb=Float64[], ub=Float64[], cpu_t=Float64[],status=String[],iter=Float64[], error=String[])
directory = "results/"
files = readdir(directory)
for filename in files
# filename = "ssd_1218114_1.out"
    # println(filename)
    instance, method, lb, ub, cpu_t, status, iter, mess = get_res(directory*filename)
    push!(df_res1, (instance, method, lb, ub, cpu_t, status, iter, mess));
end

In [ ]:
sort!(df_res1,[:inst])

In [ ]:
CSV.write("results/results.csv",df_res1)

In [ ]:
function extract_pmed(file)
    open(file,"r") do f        
        line = readline(f)
        sline = split(line)
        nJ = parse(Int64,sline[2])
        nI = parse(Int64,sline[1]) - nJ
        Icoords = Matrix{Float64}(undef, 2, nI)
        Jcoords = Matrix{Float64}(undef, 2, nJ)
        i = 1
        do_J = true
        while !eof(f)
            line = readline(f)
            sline = split(line)
            if do_J
                Jcoords[1,i] = parse(Float64,sline[1])
                Jcoords[2,i] = parse(Float64,sline[2])
                if i == nJ
                    i = 0
                    do_J = false
                end
            else
                Icoords[1,i] = parse(Float64,sline[1])
                Icoords[2,i] = parse(Float64,sline[2])
            end
            i += 1
        end
        return nI, nJ, Icoords, Jcoords
    end    
end

In [ ]:
df = DataFrame(id=Int[], in_type=String[], nI=Int[], nJ=Int[], cv=Float64[], D=Int[], FLR=Float64[], FCR=Int[])

params0 = SSD.default_params()
data = SSD.default_data()

coords_bounds=(0,100)
r_bounds=(0.3, 0.4)
k=5
t=3
a_bounds=(80,120)

id = 1

in_type = "Own"
for (nI, nJ) in [(20,3), (50,10), (70,50)]
    for cv in [0.5, 1, 1.5]
        for D in [1, 10, 25, 50, 100]
            for FLR in [0.4, 0.6, 0.8]
                for FCR in [2,4,8,10]
                    push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                    SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0)
                    id += 1
                end
            end
        end
    end
end

in_type = "p-med"
directory = "instances/p-median/"
                # (90,10)   (185,15)        (275,25)     (372,30)      (2338, 700)
for filename in ["SJC1.dat", "SJC2.dat", "SJC3a.dat", "SJC4a.dat"]#, "p3038_700.dat"]
    file = directory*filename
    nI, nJ, Icoords, Jcoords = extract_pmed(file)
    for cv in [0.5, 1, 1.5]
        for D in [1, 10, 25, 50, 100]
            for FLR in [0.4, 0.6, 0.8]
                for FCR in [2,4,8,10]
                    push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                    SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0, Icoords, Jcoords)
                    id += 1
                end
            end
        end
    end
end

In [ ]:
CSV.write("instances/instances inventory.csv",df)

In [ ]:
df = DataFrame(id=Int[], in_type=String[], nI=Int[], nJ=Int[], cv=Float64[], D=Int[], FLR=Float64[], FCR=Int[])

params0 = SSD.default_params()
data = SSD.default_data()

coords_bounds=(0,100)
r_bounds=(0.3, 0.4)
k=5
t=3
a_bounds=(80,120)

id = 1

in_type = "Own"
for (nI, nJ) in [(450, 25), (1000, 400)] #[(20,3), (70,10), (250,10)]
    for cv in [0.5, 1, 1.5]
        for D in [55000, 72500]
            for FLR in [0.6]
                for FCR in [8]
                    push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                    SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0)
                    id += 1
                end
            end
        end
    end
end

In [ ]:
# params0 = SSD.default_params()
# data = SSD.default_data()

# coords_bounds=(0,100)
# r_bounds=(0.3, 0.4)
# k=5
# t=3
# a_bounds=(80,120)

# for cv in [0.5, 1, 1.5]
#     for D in [1, 10, 25, 50, 100]
#             SSD.instance_gen(10, 3, coords_bounds, a_bounds, r_bounds, cv, D, k, t, 0.4, 2, params0)
#     end
# end

# Solve 1 inst

In [150]:
directory = "instances/Own/"
filename= "18_3.txt" #"1.txt" # "25.txt" # "6.txt"#"18_3.txt"

"18_3.txt"

In [158]:
params0 = SSD.default_params()
# params0.max_time = 60*5
data0 = SSD.default_data()

# Iterate over each instance
SSD.read_file(directory*filename, data0)

# Define the different cost levels
cost_levels = [0.60, 0.85, 1, 1.15, 1.35]
data0.F = SSD.gen_costs(data0, params0, cost_levels)
# Define the different capacity levels
cap_levels = [0.5, 0.75, 1, 1.25, 1.5]
data0.Q = SSD.gen_caps(data0, params0, cap_levels)


solve_method = "benders"#"nlp"#"heur_LS"#"iter_cuts"#"lazy_cuts"#"bendersWS"#"heur_NM"#"heur_FB" "heur_NM" - bendersPK
println("Starting $solve_method")
status = SSD.init_solver_status()
el = @CPUelapsed res = SSD.solve_ssd(data0, params0, status, solve_method)
if solve_method == "iter_cuts"
    println("$solve_method: lb=$(res[1]), ub=$(res[2]),
        of_term1_lb=$(res[3]), of_term2_lb=$(res[4]), of_term3_lb=$(res[5]),
        of_term1_ub=$(res[6]), of_term2_ub=$(res[7]), of_term3_ub=$(res[8]),
        y=$(res[9]), CPU_t=$el")
elseif solve_method in ["benders", "bendersWS1", "bendersWS2", "bendersMW", "bendersPK", "bendersSH", "bendersFC", "bendersIter","bendersIterPK"]
    println("$solve_method: of=$(res[2]), of_term1=$(res[3]), of_term2=$(res[4]), of_term3=$(res[5]), y=$(res[6]), CPU_t=$el, nodes=$(res[9]),
        feas=$(res[10]), opt=$(res[11])")
else
    println("$solve_method: of=$(res[2]), of_term1=$(res[3]), of_term2=$(res[4]), of_term3=$(res[5]), y=$(res[6]), CPU_t=$el")
end


instance file instances/Own/18_3.txt parsed successfully

Starting benders
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-18

----------------Starting iter 1 --------------------------
----- Solving SP for [4, 2, 2, 2, 2, 2, 2, 2, 2, 2] -----
Set parameter MIPGap to value 1e-05
Set parameter Threads to value 1
Set parameter TimeLimit to value 10799.098
Set parameter MIPGap to value 1e-05
Set parameter Threads to value 1
Set parameter TimeLimit to value 10799.066
Set parameter MIPGap to value 1e-05
Set parameter Threads to value 1
Set parameter TimeLimit to value 10799.033
----- Solving SP for [2, 4, 2, 2, 2, 2, 2, 2, 2, 2] -----
----- Solving SP for [2, 2, 4, 2, 2, 2, 2, 2, 2, 2] -----
----- Solving SP for [2, 2, 2, 4, 2, 2, 2, 2, 2, 2] -----
----- Solving SP for [2, 2, 2, 2, 4, 2, 2, 2, 2, 2] -----
----- Solving SP for [2, 2, 2, 2, 2, 4, 2, 2, 2, 2] -----
----- Solving SP for [2, 2, 2, 2, 2, 2, 4, 2, 2, 2] -----
----- Solving SP for [2, 2, 2, 

In [ ]:
# 1: |I| = 20 / |J| = 3
lazy_cuts: of=1176.759, of_term1=227.7617, of_term2=808.9833, of_term3=140.014, y=Any[5, 0, 3], CPU_t=1.313
benders: of=1177.7676, of_term1=227.7617, of_term2=809.3407, of_term3=140.6652, y=Any[5, 0, 3], CPU_t=5.249, nodes=1,
        feas=0, opt=28

In [ ]:
# 18_3: |I| = 250 / |J| = 10
lazy_cuts: of=7545.4853, of_term1=2397.9837, of_term2=4276.7046, of_term3=870.797, y=Any[0, 5, 4, 0, 5, 5, 3, 5, 4, 0], CPU_t=2.624

In [ ]:
# 1: |I| = 20 / |J| = 3
lazy_cuts: of=1176.759, of_term1=227.7617, of_term2=808.9833, of_term3=140.014, y=Any[5, 0, 3], CPU_t=0.063
bendersIter: of=1177.6559, of_term1=227.7617, of_term2=809.6026, of_term3=140.2916, y=Any[5, 0, 3], CPU_t=1.359, nodes=32,
        feas=0, opt=145
bendersIterPK: of=1177.6559, of_term1=227.7617, of_term2=809.6026, of_term3=140.2916, y=Any[5, 0, 3], CPU_t=1.375, nodes=32,
        feas=0, opt=145
benders: of=1177.8293, of_term1=227.7617, of_term2=809.2944, of_term3=140.7732, y=Any[5, 0, 3], CPU_t=4.046, nodes=14,
        feas=0, opt=66
bendersPK: of=1177.8354, of_term1=227.7617, of_term2=809.2969, of_term3=140.7769, y=Any[5, 0, 3], CPU_t=0.75, nodes=11,
        feas=0, opt=105

In [ ]:
bendersIter: of=8406.3404, of_term1=1951.8766, of_term2=4651.7053, of_term3=1802.7585, y=Any[0, 3, 4, 0, 4, 3, 3, 1, 4, 0], CPU_t=381.984, nodes=125,
        feas=7, opt=488

In [ ]:
# 18_3: |I| = 250 / |J| = 10
lazy_cuts: of=7545.4853, of_term1=2397.9837, of_term2=4276.7046, of_term3=870.797, y=Any[0, 5, 4, 0, 5, 5, 3, 5, 4, 0], CPU_t=2.657

In [ ]:
status

In [ ]:
bendersPK: of=7217.8524, of_term1=1762.8273, of_term2=4461.6048, of_term3=993.4203, y=Any[0, 5, 1, 0, 5, 5, 1, 1, 0, 1], CPU_t=288.984, nodes=56508,
        feas=0, opt=1691

In [ ]:
bendersPK: of=6771.5134, of_term1=1877.1471, of_term2=4230.4162, of_term3=663.9501, y=Any[0, 5, 1, 5, 2, 5, 1, 1, 1, 1], CPU_t=256.11, nodes=46107,
        feas=0, opt=1726

In [ ]:
nlp: of=7555.2371, of_term1=2397.9837, of_term2=4267.8033, of_term3=889.4501, y=Any[0, 5, 4, 0, 5, 5, 3, 5, 4, 0], CPU_t=35923.172
heur_NM: of=7555.2075, of_term1=2397.9837, of_term2=4267.7405, of_term3=889.4834, y=[0, 5, 4, 0, 5, 5, 3, 5, 4, 0], CPU_t=62.36
lazy_cuts: of=7545.4853, of_term1=2397.9837, of_term2=4276.7046, of_term3=870.797, y=Any[0, 5, 4, 0, 5, 5, 3, 5, 4, 0], CPU_t=3.157
benders: of=6771.5128, of_term1=1877.1471, of_term2=4230.4447, of_term3=663.9211, y=Any[0, 5, 1, 5, 2, 5, 1, 1, 1, 1], CPU_t=89.125, nodes=66885,
        feas=0, opt=1043

In [ ]:
289/60

In [ ]:
a=[[1,2,3], [0,0,0], [5,5,5]]

In [ ]:
# Modify the selected element
a[rand(1:length(a))] = [3,3,3]

In [ ]:
isinteger(3)

In [ ]:
a=[0.01,1,2,0.99]
sum(abs.(a-round.(a)) .<= 10e-1) == length(a)

In [ ]:
isinteger.([0,1,2,0.99])

## Is the solution correct?

In [ ]:
data0.I, data0.J, data0.k, data0.t

In [ ]:
# Allocation decisions
x = res[7]
# Location decisions
y = SSD.gen_y(data0, res[6]);

### Are the cost ok?

In [ ]:
# Fixed location cost
dot(data0.F, y)

In [ ]:
# Allocation cost
dot(data0.C, x)

In [ ]:
ρ_h = SSD.ini_ρ_h(data0) 
μ = 0

# Set-up cost
Scost = dot(data.F, y)
# Allocation cost
Allcost = 0
# Congestion cost
Congcost = 0

# Status
stats = []

primals = Dict()
for t in 1:data0.t
    primal_sp = SSD.ini_benders_sp_primal(y, data0, ρ_h, t, μ, params0, status)
    primals[t] = primal_sp
    sp_stat, sp_xval, sp_ρval, sp_Rval, sp_wval, sp_zval, sp_val, Cterm, Congterm, π1, π2, π41, π42, π61, π62, π8, π10, π11, π12 = SSD.benders_sp_primal(primal_sp, data0, ρ_h, t)
    Allcost += Cterm
    Congcost += Congterm
    append(stats, sp_stat)
end 
Scost, Allcost, Congcost

In [ ]:
SSD.calc_cost_sp(y, data0, ρ_h, primals, μ)

### Is feasible?

In [ ]:
# See if the demand can be allocated with the given capacity

In [ ]:
# For each time period, see the allocation of customers
for t in 1:data0.t
    sum(x[:,:,t]'data0.a[:,t] .> sum(y.*data0.Q,dims=2)) > 0 ? println("Bad for t=$t") : println("Ok for t=$t")
end

In [ ]:
last(sort(res[12]),10)

In [ ]:
last(sort(res[13]),10)

In [ ]:
bendersIter: of=9616.535, of_term1=1481.9548, of_term2=5378.6252, of_term3=2755.955, y=Any[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], CPU_t=71.984, nodes=100,
        feas=9, opt=342

In [ ]:
last(ordered_lb,20)

In [ ]:
# Instance Own/18_3 with bendersIter
ordered_lb = sort(res[12])
ordered_ub = sort(res[13]);

of_val = 7555.2078
of_val_rel = 5951.6929
gaps = [10e-3, 5e-2, 10e-2]#, round(abs(of_val-ordered_lb[res[end-1]])/of_val,digits=2)]
refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_lb)), label="lb",
    size=(1000,400), markershape=:xcross, legend=:topleft)

# LP relaxation optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val_rel, label="lp relax optimal", color=:orange, linestyle = :dash)
# Optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash)
# Gaps
# for i in 1:length(gaps)
#     gap = gaps[i]
#     plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash)
# end
plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

title!("I = 250, J = 10 with $solve_method - Papadakos (3h - time lim)")

# xlims!(0, 100)

In [ ]:
ordered_lb = sort(res[end-1])
ordered_ub = sort(res[end]);
# Instance Own/18_3 with benders_cb
of_val = 7555.2078
of_val_rel = 5070.6749
gaps = [10e-3, 5e-2, 10e-2, round(abs(of_val-last(collect(values(ordered_ub)))/of_val),digits=2)]
refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_ub)), label="ub",
    size=(1000,400), markershape=:xcross, legend=:topleft)

# LP relaxation optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val_rel, label="lp relax optimal", color=:orange, linestyle = :dash)
# Optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash)
# Gaps
# for i in 1:length(gaps)
#     gap = gaps[i]
#     plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash)
# end
# plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

title!("I = 250, J = 10 with $solve_method - Papadakos (163 s)")

xlims!(0, 100)

In [ ]:
# Instance Own/6
of_val = 1914.4834
gaps = [10e-3, 5e-2, 10e-2, round(abs(of_val-ordered_lb[res[end-1]])/of_val,digits=2)]
refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_lb)), label="lb",
    size=(1000,400), markershape=:xcross)
plot!(1:status.nIter, collect(values(ordered_ub)), label="ub", markershape=:xcross)
# ylims!(1500, 4000)
# Optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash, lw=1)
# Gaps
for i in 1:length(gaps)
    gap = gaps[i]
    plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash, lw=1)
    annotate!(status.nIter, refs[i], text("$gap", :left, 12, :black))
end
plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

title!("I = 20, J = 3")

In [ ]:
plot(1:status.nIter, [collect(values(ordered_lb)) collect(values(ordered_ub))], label=["lb" "ub"],
    size=(1000,400), markershape=:xcross)
# ylims!(1500, 4000)
plot!(1:status.nIter, zeros(status.nIter).+1914.4834) #3649.0341)

In [ ]:
# Own 9
lazy_cuts: of=1914.4834, of_term1=660.1081, of_term2=828.7458, of_term3=425.6295, y=Any[5, 3, 0], CPU_t=0.234
benders: of=1959.8316, of_term1=801.0565, of_term2=858.3784, of_term3=300.3967, y=Any[5, 5, 0], CPU_t=0.141, nodes=3,
        feas=0, opt=40

# Own 8
benders: of=5510.3143, of_term1=1661.3185, of_term2=2132.8105, of_term3=1716.1852, y=Any[1, 0, 0, 0, 1, 5, 4, 0, 4, 0], CPU_t=6.312, nodes=6936,
        feas=21, opt=243
lazy_cuts: of=3649.0341, of_term1=1705.6408, of_term2=1523.3316, of_term3=420.0617, y=Any[0, 0, 0, 0, 3, 4, 4, 4, 0, 5], CPU_t=31.703



In [ ]:
x=[-3.6979166666666665 4.697916666666666 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 0.0 1.0; 0.0 0.0 1.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 -9.634408602150538 10.634408602150538; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0]

In [ ]:
lazy_cuts: of=895.9974, of_term1=394.1035, of_term2=501.3773, of_term3=0.5166, y=Any[1, 1, 3], CPU_t=1.375

benders: of=949.9676, of_term1=448.7567, of_term2=501.0648, of_term3=0.1462, y=Any[1, 1, 5], CPU_t=0.249, nodes=17,
        feas=15, opt=30

In [ ]:
y = [-0.0 -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0 -0.0]
x = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 0.0 1.0; 0.0 0.0 1.0; 0.0 0.0 1.0; 1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 1.0 0.0; 0.0 0.0 1.0; 1.0 0.0 0.0];

In [ ]:
dot(data0.F, y)

In [ ]:
sum(data0.C[i,j,t]*x[i,j] for i in 1:data0.I for j in 1:data0.J)

In [ ]:
# C1
for j in 1:data0.J
    println("$j: $(-sum(data0.a[i,t]*x[i,j] for i in 1:data0.I)) >= $(-sum(data0.Q[j,k]*y[j,k] for k in 1:data0.k))")
end

In [ ]:
for j in 1:data0.J
    println("$j: $(sum(data0.a[i,t]*x[i,j] for i in 1:data0.I))")
end

In [ ]:
# Pi 1 a 12
benders: of=604.2782, of_term1=0.0, of_term2=604.2775, of_term3=0.0008, y=Any[0, 0, 0], CPU_t=2.672, nodes=1,
        feas=0, opt=15
# Pi 9
benders: of=1130.4119, of_term1=0.0, of_term2=1130.4144, of_term3=-0.0024, y=Any[0, 0, 0], CPU_t=2.297, nodes=0,
        feas=15, opt=0

In [ ]:
# 600 new valid inequality
benders: of=74561.0501, of_term1=23586.6244, of_term2=50953.9516, of_term3=20.4741, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=8.249, nodes=2048,
        feas=0, opt=471
bendersWS: of=74561.0434, of_term1=23586.6182, of_term2=50953.9944, of_term3=20.4309, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=6.421, nodes=3282,
        feas=0, opt=391
bendersMW: of=74561.0482, of_term1=23586.6244, of_term2=50953.9304, of_term3=20.4934, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=17.235, nodes=874,
        feas=0, opt=618
bendersPK: of=74561.0467, of_term1=23586.6244, of_term2=50953.9454, of_term3=20.4769, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=5.829, nodes=871,
        feas=0, opt=611
bendersSH: of=74560.9944, of_term1=23586.6246, of_term2=50953.8125, of_term3=20.5574, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=6.906, nodes=3031,
        feas=0, opt=578
bendersFC: of=74561.0496, of_term1=23586.6244, of_term2=50953.9287, of_term3=20.4965, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=7.344, nodes=855,
        feas=0, opt=189

# No new VI
benders: of=74561.0501, of_term1=23586.6243, of_term2=50953.9091, of_term3=20.5168, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=5.828, nodes=5700,
        feas=0, opt=548
bendersWS: of=74561.0497, of_term1=23586.6246, of_term2=50953.8344, of_term3=20.5907, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=7.844, nodes=5364,
        feas=0, opt=561
bendersMW: of=74561.0501, of_term1=23586.6245, of_term2=50953.9612, of_term3=20.4644, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=10.719, nodes=1572,
        feas=0, opt=470
bendersPK: of=74561.0506, of_term1=23586.6244, of_term2=50953.9611, of_term3=20.4652, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=6.687, nodes=1203,
        feas=0, opt=629
bendersSH: of=74560.9951, of_term1=23586.6245, of_term2=50953.832, of_term3=20.5386, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=10.155, nodes=5851,
        feas=0, opt=870
bendersFC: of=74561.0501, of_term1=23586.6244, of_term2=50953.9124, of_term3=20.5133, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=11.922, nodes=1040,
        feas=3, opt=222

In [ ]:
# 253
benders: of=2136.2163, of_term1=761.9652, of_term2=1370.4704, of_term3=3.7807, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=5.125
bendersWS: of=2136.2164, of_term1=761.9652, of_term2=1370.4708, of_term3=3.7804, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=7.782
bendersMW: of=2136.2163, of_term1=761.9652, of_term2=1370.4711, of_term3=3.78, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=27.078
bendersPK: of=2136.2163, of_term1=761.9652, of_term2=1370.4725, of_term3=3.7786, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=4.36
bendersSH: of=2136.2154, of_term1=761.9652, of_term2=1370.4701, of_term3=3.7801, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=6.938
bendersFC: of=2136.2157, of_term1=761.9652, of_term2=1370.4698, of_term3=3.7807, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=8.531

In [ ]:
# 600
benders: of=74561.0501, of_term1=23586.6243, of_term2=50953.8793, of_term3=20.5466, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=4.531
bendersWS: of=74561.0489, of_term1=23586.6242, of_term2=50953.9102, of_term3=20.5145, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=5.64
bendersPK: of=74561.0492, of_term1=23586.6244, of_term2=50953.9351, of_term3=20.4897, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=7.265
bendersSH: of=74560.9948, of_term1=23586.6253, of_term2=50953.9062, of_term3=20.4632, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=5.75
bendersFC: of=74561.0505, of_term1=23586.6244, of_term2=50953.9369, of_term3=20.4892, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=10.453
bendersMW: of=74561.0493, of_term1=23586.6247, of_term2=50953.9446, of_term3=20.4799, y=Any[0, 0, 5, 5, 5, 5, 1, 0, 0, 0], CPU_t=12.843

In [ ]:
# OF mp, OF sum
res[1], res[2]

In [ ]:
# 74.561.0497
res[3]+ res[4] + res[5]

In [ ]:
res[3], res[4] , res[5]

In [ ]:
#235
res[9], res[10], res[11]

In [ ]:
#600
res[9], res[10], res[11]

In [ ]:
#600
res[9], res[10], res[11]

In [ ]:
#235
res[9], res[10], res[11]

In [ ]:
#235
res[9], res[10], res[11]

In [ ]:
#235
res[9], res[10], res[11]

In [ ]:
#235
res[9], res[10], res[11]

In [ ]:
# 404
res[9], res[10], res[11]

In [ ]:
#130 
lazy_cuts: of=895.9974, of_term1=394.1035, of_term2=501.3766, of_term3=0.5173, y=Any[1, 1, 3], CPU_t=0.046
benders: of=895.9974, of_term1=394.1035, of_term2=501.3747, of_term3=0.5191, y=Any[1, 1, 3], CPU_t=0.157
# 200
lazy_cuts: of=2301.6885, of_term1=1333.1643, of_term2=966.8477, of_term3=1.6765, y=Any[0, 0, 4, 5, 0, 0, 0, 1, 4, 1], CPU_t=11.296
benders: of=2301.6881, of_term1=1333.1643, of_term2=966.8512, of_term3=1.673, y=Any[0, 0, 4, 5, 0, 0, 0, 1, 4, 1], CPU_t=8.094
#235
nlp: of=2136.2173, of_term1=761.9652, of_term2=1370.498, of_term3=3.754, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=43.626
lazy_cuts: of=2136.2164, of_term1=761.9652, of_term2=1370.4727, of_term3=3.7785, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=3.532
lazy_cuts: of=2136.2163, of_term1=761.9652, of_term2=1370.4722, of_term3=3.7789, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=10.0
benders: of=2136.2154, of_term1=761.9652, of_term2=1370.4699, of_term3=3.7811, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=2.953
bendersSH: of=2076.9208, of_term1=761.9652, of_term2=1313.2436, of_term3=1.7122, y=Any[1, 0, 0, 0, 0, 5, 5, 0, 5, 0], CPU_t=2.937

# 404
lazy_cuts: of=6473.8752, of_term1=5017.8601, of_term2=1441.3593, of_term3=14.6558,
y=Any[5, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 5, 1, 0, 0, 5, 0, 5, 0, 5, 0, 5, 0, 0, 5, 
    5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 4], CPU_t=469.5

benders: of=6312.4219, of_term1=4879.4955, of_term2=1466.0399, of_term3=454.4843,
y=Any[5, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 5, 0, 5, 0, 5, 0, 5, 0, 0, 5,
    5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 5], CPU_t=2559.781

In [ ]:
xlc = res[7]
ylc = res[6];

In [ ]:
xben = res[7]
yben = res[6];

In [ ]:
# Are equal?
xben == xlc

In [ ]:
# Assignation cost
dot(data0.C,xben[:,:,:]), dot(data0.C,xlc[:,:,:])

In [ ]:
t=1

In [ ]:
# Demand per facility Bend
demben = sum(data0.a[:,t] .* xben[:,:,t], dims = 1)

In [ ]:
# Cap available Benders
capben = sum(data0.Q .* SSD.gen_y(data0,yben), dims=2)'

In [ ]:
demben .<= capben

In [ ]:
# Demand per facility Lazy Cuts
demlc = sum(data0.a[:,t] .* xlc[:,:,t], dims = 1)

In [ ]:
# Cap available Lazy
caplc = sum(data0.Q .* SSD.gen_y(data0,ylc), dims=2)'

In [ ]:
demlc .<= caplc

In [ ]:
ini_ρ_h = collect(range(0.1,step=0.1,0.9))
H = 1:length(ini_ρ_h)
ρ_h = Array{Float64}(undef,data0.J,data0.t,length(ini_ρ_h))

for j in 1:data0.J, t in 1:data0.t, h in H
    ρ_h[j, t, h]= ini_ρ_h[h]
end

# Are all the methods correct?

See if the x are feasibles

In [ ]:
SSD.gen_y(data0, res[6])

In [ ]:
sum(data0.Q .* SSD.gen_y(data0, res[6]), dims=2)

In [ ]:
t = 1
mat = data0.a[:,t] .* res[7].data[:,:,t]
# mat = data0.a[:,t] .* res[7][:,:,t]
sum(mat, dims=1)

In [ ]:
# X benders
x_benders = res[7]

In [ ]:
# X lazy cuts
x_lazy_cuts = res[7].data

In [ ]:
# X lazy cuts vs benders
t = 3
x_lazy_cuts[:,:,t].==x_benders[:,:,t]

In [ ]:
x_lazy_cuts[:,:,t]

In [ ]:
x_benders[:,:,t]

In [ ]:
# Iterative cuts

In [ ]:
sum(data0.Q .* SSD.gen_y(data0, res[9]), dims=2)

In [ ]:
t = 1
mat = data0.a[:,t] .* res[10].data[:,:,t]
sum(mat, dims=1)

In [ ]:
repeat(["Own"],4)

# Solver several instances for several methods

In [ ]:
directory = "instances/Own/"
filename="130.txt"
"""
Type    I    J
Own
130     20   3
235     50   10
404     70   50

p-med
600     90   10    
"""

# instances = ["instances/Own/130.txt", "instances/Own/235.txt", "instances/Own/404.txt", "instances/p-med/550.txt"]
instances = ["130.txt", "235.txt", "404.txt", "600.txt"]
instances = ["13.txt","14.txt","15.txt","16.txt","17.txt","18.txt"]
instances = ["18_3.txt"] #, "18_2.txt", "18_3.txt"]
# instances = ["130.txt"]
directories = repeat(["instances/Own/"],length(instances)) #["instances/Own/", "instances/Own/", "instances/Own/"]
results = []
results_benders = []

col_names_res = ["I","J","cv","D","instance","method","of", "FC", "AC", "CC", "y","CPU_t_s", "status"]
push!(results, [i for i in col_names_res])

col_names_benders = ["I","J","cv","D","instance","method","of", "FC", "AC", "CC","y","nodes","f_cut","opt_cut","CPU_t_s", "status"]
push!(results_benders, [i for i in col_names_benders])

for (instance, directory) in zip(instances,directories)

    params0 = SSD.default_params()
    data0 = SSD.default_data()
    
    # Iterate over each instance
    SSD.read_file(directory*instance, data0)
    
    # Define the different cost levels
    cost_levels = [0.60, 0.85, 1, 1.15, 1.35]
    data0.F = SSD.gen_costs(data0, params0, cost_levels)
    # Define the different capacity levels
    cap_levels = [0.5, 0.75, 1, 1.25, 1.5]
    data0.Q = SSD.gen_caps(data0, params0, cap_levels)
    
    for solve_method in ["bendersIter"]#["iter_cuts","lazy_cuts","benders", "bendersPK","bendersIter"]#["iter_cuts","lazy_cuts","benders","bendersWS1","bendersWS2", "bendersMW", "bendersPK", "bendersSH", "bendersFC", "heur_NM", "heur_FB"]#["nlp", "iter_cuts", "lazy_cuts", "benders", "bendersWS","bendersMW", "bendersPK", "bendersSH", "heur_NM", "heur_FB"]#
        println("")
        println("Starting $solve_method")
        status = SSD.init_solver_status()
        try
            el = @CPUelapsed res = SSD.solve_ssd(data0, params0, status, solve_method)
            println("$solve_method: of=$(res[1]), of_term1=$(res[3]), of_term2=$(res[4]), of_term3=$(res[5]), y=$(res[6])")
            if solve_method == "iter_cuts"
                push!(results, (data0.I, data0.J, data0.cv, data0.D, instance, solve_method, ("$(res[1]) - $(res[2])"), ("$(res[3]) - $(res[6])"), ("$(res[4]) - $(res[7])"), ("$(res[5]) - $(res[8])"), "$([i for i in res[9]])", el, res[11]))
            else
                push!(results, (data0.I, data0.J, data0.cv, data0.D, instance, solve_method, res[1], res[3], res[4], res[5], "$([i for i in res[6]])", el, res[8]))
            end
            if solve_method in ["benders", "bendersWS", "bendersMW", "bendersPK", "bendersSH", "bendersFC","bendersIter"]
                push!(results_benders, (data0.I, data0.J, data0.cv, data0.D, instance, solve_method, res[1], res[3], res[4], res[5], "$([i for i in res[6]])", res[9], res[10], res[11], el, res[8]))
            end
        catch e
            push!(results, (data0.I, data0.J, data0.cv, data0.D, instance, solve_method, "error", "", "", "", "", "", ""))
        end
        println(status.endStatus)
    end
end

In [ ]:
ub = 10967.009707
lb = 5096.226159395523 
(ub-lb)/ub

In [ ]:
cols = [collect(results[i]) for i in eachindex(results)]
df_res = DataFrame(cols, [Symbol(i) for i in eachindex(cols)]);
# CSV.write("240802_res_big2.csv", df_res)

In [ ]:
df_res

In [ ]:
df_res

In [ ]:
cols_ben = [collect(results_benders[i]) for i in eachindex(results_benders)]
df_res_ben = DataFrame(cols_ben, [Symbol(i) for i in eachindex(cols_ben)]);
CSV.write("240723_res_ins 8_benders.csv", df_res_ben)

In [ ]:
df_res_ben